In [1]:
import pandas as pd
import altair as alt

In [2]:
def patreon_fee(amt):
    if amt > 3.00:
        return 0.30 + 0.029 * amt
    else:
        return 0.10 + 0.050 * amt
    
def patreon_paid(amt):
    return round(amt - patreon_fee(amt), 2)

def patreon_mrr(plan, subscribers, paid):
    match plan:
        case "lite":
            platform_fee = (subscribers * paid) * 0.05
        case "pro":
            platform_fee = (subscribers * paid) * 0.08
        case "premium":
            platform_fee = (subscribers * paid) * 0.12
    income = sum(map(lambda x: patreon_paid(paid), range(subscribers)))
    return round(income - platform_fee, 2)

In [3]:
patreon_paid(3.00), patreon_mrr("lite", 100, 3.00), patreon_mrr("pro", 100, 3.00), patreon_mrr("premium", 100, 3.00)

(2.75, 260.0, 251.0, 239.0)

In [4]:
patreon_paid(5.00), patreon_mrr("lite", 100, 5.00), patreon_mrr("pro", 100, 5.00), patreon_mrr("premium", 100, 5.00)

(4.55, 430.0, 415.0, 395.0)

In [5]:
def memberful_fee(plan, amt):
    match plan:
        case "starter":
            return 0.30 + 0.029 * amt + 0.100 * amt
        case "pro" | "premium":
            return 0.30 + 0.029 * amt + 0.049 * amt
        
def memberful_paid(plan, amt):
    return round(amt - memberful_fee(plan, amt), 2)

def memberful_mrr(plan, subscribers, paid):
    match plan:
        case "starter":
            platform_fee = 0.00
        case "pro":
            platform_fee = 25.00
        case "premium":
            platform_fee = 100.00
    income = sum(map(lambda x: memberful_paid(plan, paid), range(subscribers)))
    return round(income - platform_fee, 2)

In [6]:
memberful_paid("starter", 3.00), memberful_mrr("starter", 100, 5.00), memberful_mrr("pro", 100, 5.00), memberful_mrr("premium", 100, 5.00)

(2.31, 405.0, 406.0, 331.0)

In [7]:
def kofi_fee(plan, amt):
    match plan:
        case "contributor":
            return 0.30 + 0.029 * amt + 0.05 * amt
        case "gold":
            return 0.30 + 0.029 * amt
        
def kofi_paid(plan, amt):
    return round(amt - kofi_fee(plan, amt), 2)

def kofi_mrr(plan, subscribers, paid):
    match plan:
        case "contributor":
            platform_fee = 0.00
        case "gold":
            platform_fee = 6.00
    income = sum(map(lambda x: kofi_paid(plan, paid), range(subscribers)))
    return round(income - platform_fee, 2)

In [8]:
def stripe_fee(plan, amt):
    match plan:
        case "starter":
            return 0.05 * amt
        case "scale":
            return 0.08 * amt
        
def stripe_paid(plan, amt):
    return round(amt - stripe_fee(plan, amt), 2)

def stripe_mrr(plan, subscribers, paid):
    platform_fee = 0.00
    income = sum(map(lambda x: stripe_paid(plan, paid), range(subscribers)))
    return round(income - platform_fee, 2)

In [9]:
stripe_paid("starter", 3.00), stripe_mrr("starter", 100, 5.00), stripe_mrr("scale", 100, 5.00)

(2.85, 475.0, 460.0)

In [10]:
patreon = []
for n in range(0,1025,25):
    patreon.append({
        'platform':'patreon-lite', 
        'plan':'lite',
        'subscribers': n,
        'mrr': patreon_mrr('lite', n, 5.00)
    })
    patreon.append({
        'platform':'patreon-pro', 
        'plan':'pro',
        'subscribers': n,
        'mrr': patreon_mrr('pro', n, 5.00)
    })
    patreon.append({
        'platform':'patreon-premium', 
        'plan':'premium',
        'subscribers': n,
        'mrr': patreon_mrr('premium', n, 5.00)
    })

In [11]:
memberful = []
for n in range(0,1025,25):
    memberful.append({
        'platform':'memberful-starter', 
        'plan':'starter',
        'subscribers': n,
        'mrr': memberful_mrr('starter', n, 5.00)
    })
    memberful.append({
        'platform':'memberful-pro', 
        'plan':'pro',
        'subscribers': n,
        'mrr': memberful_mrr('pro', n, 5.00)
    })
    memberful.append({
        'platform':'memberful-premium', 
        'plan':'premium',
        'subscribers': n,
        'mrr': memberful_mrr('premium', n, 5.00)
    })

In [12]:
kofi = []
for n in range(0,1025,25):
    kofi.append({
        'platform':'kofi-contributor', 
        'plan':'contributor',
        'subscribers': n,
        'mrr': kofi_mrr('contributor', n, 5.00)
    })
    kofi.append({
        'platform':'kofi-gold', 
        'plan':'gold',
        'subscribers': n,
        'mrr': kofi_mrr('gold', n, 5.00)
    })

In [13]:
stripe = []
for n in range(0,1025,25):
    stripe.append({
        'platform':'stripe-starter', 
        'plan':'starter',
        'subscribers': n,
        'mrr': stripe_mrr('starter', n, 5.00)
    })
    stripe.append({
        'platform':'stripe-scale', 
        'plan':'scale',
        'subscribers': n,
        'mrr': stripe_mrr('scale', n, 5.00)
    })

In [24]:
all_platforms = pd.DataFrame(patreon + kofi + memberful + stripe)

In [23]:
plt = alt.Chart(
    all_platforms
).encode(
    x="subscribers",
    y="mrr",
    color="platform",
    tooltip=["platform", "mrr"]
).properties(
    width=1024,
    height=768
)
plt.mark_line() + plt.mark_circle()

alt.LayerChart(...)

## Pricing Links

- [Patreon](https://www.patreon.com/pricing)
- [Memberful](https://memberful.com/pricing/)
- [Ko-Fi](https://ko-fi.com/gold)
- [Stripe Billing](https://stripe.com/billing/pricing)